In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyathena import connect
from pyathena.pandas_cursor import PandasCursor

#from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import warnings

#warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 30
pd.options.display.max_rows = 50

# Define connection to DB
conn = connect(
    s3_staging_dir='s3://aws-athena-query-results-323906537337-us-east-1/',
    region_name='us-east-1',
    cursor_class=PandasCursor
    )
cursor = conn.cursor()

# 0. Data

## 0.1 pbp data

In [2]:
# a work around -- issue with QB parsing for each play, since a snap player could be other than a QB
#
# playtypeid:
# 1 = pass attempt
# 2 = incomplete pass attempt
# 3 = sack
# 4 = designed rush OR scramble
# 9 = intercepted pass attempt

simple_query = f'''
select
    cast(season as integer) season,
    eventmetadata.week, eventmetadata.gamecode, eventmetadata.eventtypeid, eventmetadata.gamedateutcepoch gameTime,
    if(teammetadata[1].ishometeam, teammetadata[1].teamid, teammetadata[2].teamid) homeTeam,
    if(teammetadata[2].ishometeam, teammetadata[1].teamid, teammetadata[2].teamid) awayTeam,
    
    playid, pbp.driveid, pbp.period,  
    cast(secondsremaininginperiod as double) secondsRemainingInPeriod, 
    
    pbp.startpossessionteamid, pbp.endpossessionteamid,
    if(pbp.startpossessionteamid = teammetadata[1].teamid, teammetadata[2].teamid, teammetadata[1].teamid) defenseTeam,
    
    pbp.down, 
    cast(pbp.distance as double) yardsToGo,
    cast(pbp.startyardsfromgoal as double) startYardsFromGoal, 
    cast(pbp.endyardsfromgoal as double) endYardsFromGoal,
    pbp.awayscorebefore, pbp.awayscoreafter, pbp.homescorebefore, pbp.homescoreafter, 
    pbp.iscontinuation, 
    -- pbp.penaltytype.penaltytypeid, pbp.penaltytype.name, 
    pbp.playtype.playtypeid, pbp.playtype.name playName,
    
    -- parse info for QB
    xinfo.playersnappedto playerid,
    
    xinfo.rolloutlocation.abbreviation rolloutlocation,
    xinfo.handofftype.abbreviation handofftype
from 
    datalakefootball.pbp_xinfo_enriched
where 
    season>='2015' and 
    eventmetadata.eventtypeid in (1,2) and 
    pbp.period <= 4 and
    pbp.down is not null and 
    pbp.playtype.playtypeid in (1,2,3,4,9)
order by season, eventmetadata.week, eventmetadata.gamecode, pbp.period, playid, secondsremaininginperiod desc
'''

if True:
    pbp_df = cursor.execute(simple_query).as_pandas()
    print(pbp_df.info())
else:
    print("Failed to query!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165439 entries, 0 to 165438
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   season                    165439 non-null  Int64  
 1   week                      165439 non-null  Int64  
 2   gamecode                  165439 non-null  Int64  
 3   eventtypeid               165439 non-null  Int64  
 4   gameTime                  165439 non-null  Int64  
 5   homeTeam                  165439 non-null  Int64  
 6   awayTeam                  165439 non-null  Int64  
 7   playid                    165439 non-null  float64
 8   driveid                   165439 non-null  Int64  
 9   period                    165439 non-null  Int64  
 10  secondsRemainingInPeriod  165439 non-null  float64
 11  startpossessionteamid     165439 non-null  Int64  
 12  endpossessionteamid       165439 non-null  Int64  
 13  defenseTeam               165439 non-null  I

In [3]:
# remove rows without player id
pbp_df = pbp_df[~pbp_df.playerid.isnull()]

# create score difference: offenseTeam - defenseTeam
pbp_df['scoreDiff'] = pbp_df.homescorebefore - pbp_df.awayscorebefore

id = (pbp_df.startpossessionteamid == pbp_df.awayTeam).tolist()

pbp_df.loc[id, 'scoreDiff'] = -pbp_df.loc[id, 'scoreDiff']

pbp_df.scoreDiff = pbp_df.scoreDiff.astype('float64')

#pbp_df.down = pbp_df.down.astype('float64')

pbp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165121 entries, 0 to 165299
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   season                    165121 non-null  Int64  
 1   week                      165121 non-null  Int64  
 2   gamecode                  165121 non-null  Int64  
 3   eventtypeid               165121 non-null  Int64  
 4   gameTime                  165121 non-null  Int64  
 5   homeTeam                  165121 non-null  Int64  
 6   awayTeam                  165121 non-null  Int64  
 7   playid                    165121 non-null  float64
 8   driveid                   165121 non-null  Int64  
 9   period                    165121 non-null  Int64  
 10  secondsRemainingInPeriod  165121 non-null  float64
 11  startpossessionteamid     165121 non-null  Int64  
 12  endpossessionteamid       165121 non-null  Int64  
 13  defenseTeam               165121 non-null  I

In [ ]:
### testing
print('pass         ', sum(pbp_df.playtypeid.isin([1,2,9])) / len(pbp_df.playtypeid))
print('rush/scramble', sum(pbp_df.playtypeid.isin([4])) / len(pbp_df.playtypeid))
print('sack         ', sum(pbp_df.playtypeid.isin([3])) / len(pbp_df.playtypeid) )

# 1. Feature / label preparation

In [4]:
label_pass = pbp_df.playtypeid.isin([1,2,9])

label_sack = pbp_df.playtypeid.isin([3])

label_rush = pbp_df.playtypeid.isin([4]) & (pbp_df.rolloutlocation=='N')

label_scramble = pbp_df.playtypeid.isin([4]) & (pbp_df.handofftype=='Q') & (pbp_df.rolloutlocation!='N')

In [14]:
label_pass.dtype

dtype('bool')

In [5]:
num_fields = [
                'scoreDiff',
                'yardsToGo',
                'startYardsFromGoal',
                'secondsRemainingInPeriod',
             ]

cat_fields = [
                #'eventtypeid',
                'period',
                'down',
             ]
              
# StandardScaler version
# StandardScaler should not be applied to testing data
transform_pipeline = ColumnTransformer(transformers=[
                                            ('num', StandardScaler(), num_fields),
                                            ('cat', OneHotEncoder(categories='auto'), cat_fields)
                                        ])
features_transformed = transform_pipeline.fit_transform(pbp_df)

feature_names = num_fields.copy()
cat_one_hot_fields = list(transform_pipeline.named_transformers_.cat.get_feature_names(input_features=cat_fields))
feature_names.extend(cat_one_hot_fields)

if type(features_transformed) == np.ndarray:
    features_transformed = pd.DataFrame(features_transformed, columns=feature_names)
else:
    features_transformed = pd.DataFrame(features_transformed.toarray(), columns=feature_names)


# None-StandardScaler version
transform_pipeline_2 = ColumnTransformer(transformers=[
                                            ('num', 'passthrough', num_fields),
                                            ('cat', OneHotEncoder(categories='auto'), cat_fields)
                                        ])
features_transformed_2 = transform_pipeline_2.fit_transform(pbp_df)

if type(features_transformed_2) == np.ndarray:
    features_transformed_2 = pd.DataFrame(features_transformed_2, columns=feature_names)
else:
    features_transformed_2 = pd.DataFrame(features_transformed_2.toarray(), columns=feature_names)

features_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165121 entries, 0 to 165120
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   scoreDiff                 165121 non-null  float64
 1   yardsToGo                 165121 non-null  float64
 2   startYardsFromGoal        165121 non-null  float64
 3   secondsRemainingInPeriod  165121 non-null  float64
 4   period_1                  165121 non-null  float64
 5   period_2                  165121 non-null  float64
 6   period_3                  165121 non-null  float64
 7   period_4                  165121 non-null  float64
 8   down_1                    165121 non-null  float64
 9   down_2                    165121 non-null  float64
 10  down_3                    165121 non-null  float64
 11  down_4                    165121 non-null  float64
dtypes: float64(12)
memory usage: 15.1 MB


In [6]:
features_transformed_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165121 entries, 0 to 165120
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   scoreDiff                 165121 non-null  float64
 1   yardsToGo                 165121 non-null  float64
 2   startYardsFromGoal        165121 non-null  float64
 3   secondsRemainingInPeriod  165121 non-null  float64
 4   period_1                  165121 non-null  float64
 5   period_2                  165121 non-null  float64
 6   period_3                  165121 non-null  float64
 7   period_4                  165121 non-null  float64
 8   down_1                    165121 non-null  float64
 9   down_2                    165121 non-null  float64
 10  down_3                    165121 non-null  float64
 11  down_4                    165121 non-null  float64
dtypes: float64(12)
memory usage: 15.1 MB


# 2. Model study

In [8]:
# instantiate models
folds = 5

model_logistic = LogisticRegression(solver='lbfgs', fit_intercept=True, max_iter=10000, tol=1e-4, random_state=42)

model_SGD = SGDClassifier(loss='log', penalty=None, fit_intercept=True, max_iter=10000, tol=1e-4, random_state=42)

model_SVC = LinearSVC(max_iter=10000, tol=1e-4, random_state=42)

model_RF = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)

id_train = (pbp_df.season <= 2018).tolist()

In [ ]:
# Logistic regression model

scores_logistic = cross_validate(model_logistic,
    features_transformed[id_train],
    label_pass[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Pass - Linear logistic regression (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_logistic['test_recall'].mean(), 
                                                                                   scores_logistic['test_precision'].mean(), 
                                                                                   scores_logistic['test_f1'].mean(),
                                                                                  scores_logistic['test_roc_auc'].mean()))

scores_logistic = cross_validate(model_logistic,
    features_transformed[id_train],
    label_rush[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Rush - Linear logistic regression (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_logistic['test_recall'].mean(), 
                                                                                   scores_logistic['test_precision'].mean(), 
                                                                                   scores_logistic['test_f1'].mean(),
                                                                                  scores_logistic['test_roc_auc'].mean()))

# for imbalanced event, only roc_auc para makes sense
scores_logistic = cross_validate(model_logistic,
    features_transformed[id_train],
    label_sack[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Sack - Linear logistic regression (P/R/F1/ROC): {:.1%}'.format(scores_logistic['test_roc_auc'].mean()))

scores_logistic = cross_validate(model_logistic,
    features_transformed[id_train],
    label_scramble[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Scramble - Linear logistic regression (P/R/F1/ROC): {:.1%}'.format(scores_logistic['test_roc_auc'].mean()))

In [ ]:
# SVM model

scores_SVC = cross_validate(model_SVC,
    features_transformed[id_train],
    label_pass[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Pass - SVC Classifier (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_SVC['test_recall'].mean(), 
                                                                        scores_SVC['test_precision'].mean(), 
                                                                        scores_SVC['test_f1'].mean(),
                                                                        scores_SVC['test_roc_auc'].mean()))

scores_SVC = cross_validate(model_SVC,
    features_transformed[id_train],
    label_rush[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Rush - SVC Classifier (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_SVC['test_recall'].mean(), 
                                                                        scores_SVC['test_precision'].mean(), 
                                                                        scores_SVC['test_f1'].mean(),
                                                                        scores_SVC['test_roc_auc'].mean()))

# for imbalanced event, only roc_auc para makes sense
scores_SVC = cross_validate(model_SVC,
    features_transformed[id_train],
    label_sack[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Sack - SVC Classifier (P/R/F1/ROC): {:.1%}'.format(scores_SVC['test_roc_auc'].mean()))

scores_SVC = cross_validate(model_SVC,
    features_transformed[id_train],
    label_scramble[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Scramble - SVC Classifier (P/R/F1/ROC): {:.1%}'.format(scores_SVC['test_roc_auc'].mean()))

In [9]:
scores_RF = cross_validate(model_RF,
    features_transformed_2[id_train],
    label_pass[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Pass - Random Forest Classifier (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_RF['test_recall'].mean(), 
                                                                                scores_RF['test_precision'].mean(), 
                                                                                scores_RF['test_f1'].mean(),
                                                                                scores_RF['test_roc_auc'].mean()))

Pass - Random Forest Classifier (P/R/F1/ROC): 71.5%  71.2%  71.3% 74.3%


In [13]:
label_pass

0         False
1          True
2         False
3          True
4         False
          ...  
165295    False
165296     True
165297     True
165298     True
165299     True
Name: playtypeid, Length: 165121, dtype: bool

In [ ]:
# Random forest

scores_RF = cross_validate(model_RF,
    features_transformed_2[id_train],
    label_pass[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Pass - Random Forest Classifier (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_RF['test_recall'].mean(), 
                                                                                scores_RF['test_precision'].mean(), 
                                                                                scores_RF['test_f1'].mean(),
                                                                                scores_RF['test_roc_auc'].mean()))

scores_RF = cross_validate(model_RF,
    features_transformed_2[id_train],
    label_rush[id_train],
    cv=folds,
    scoring=(['recall','precision','f1','roc_auc']))
print('Rush - Random Forest Classifier (P/R/F1/ROC): {:.1%}  {:.1%}  {:.1%} {:.1%}'.format(scores_RF['test_recall'].mean(), 
                                                                                scores_RF['test_precision'].mean(), 
                                                                                scores_RF['test_f1'].mean(),
                                                                                scores_RF['test_roc_auc'].mean()))

# for imbalanced event, only roc_auc para makes sense
scores_RF = cross_validate(model_RF,
    features_transformed_2[id_train],
    label_sack[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Sack - Random Forest Classifier (P/R/F1/ROC): {:.1%}'.format(scores_RF['test_roc_auc'].mean()))

scores_RF = cross_validate(model_RF,
    features_transformed[id_train],
    label_scramble[id_train],
    cv=folds,
    scoring=(['roc_auc']))
print('Scramble - Random Forest Classifier (P/R/F1/ROC): {:.1%}'.format(scores_RF['test_roc_auc'].mean()))

In [ ]:
# Check prediction average against observation average

id = (pbp_df.season <= 2018).tolist()
id_test = (pbp_df.season >= 2019).tolist()

# pass
model_logistic.fit(features_transformed[id], label_pass[id])
re = model_logistic.predict(features_transformed[id])
re_prob_pass = model_logistic.predict_proba(features_transformed[id])

# create output file
outmat = pbp_df.loc[id,['season','week','gamecode','eventtypeid','period','playid','secondsRemainingInPeriod','startpossessionteamid','down','yardsToGo','startYardsFromGoal','scoreDiff']]
outmat['isPass'] = label_pass[id]
outmat['pred_pass_prob'] = re_prob_pass[:,1]
outmat.to_csv('QB_pass_training_data.csv', index=False)

print("QB fitted average pass ratio: {:.2%}".format(sum(re)/len(re)) )
print("QB fitted average pass_prob ratio: {:.2%}".format(np.mean(re_prob_pass[:,1])) )
print("QB actual average pass ratio: {:.2%}\n".format(sum(label_pass[id])/sum(id)) )

# rush
model_logistic.fit(features_transformed[id], label_rush[id])
re = model_logistic.predict(features_transformed[id])
re_prob_rush = model_logistic.predict_proba(features_transformed[id])

# create output file
outmat = pbp_df.loc[id,['season','week','gamecode','eventtypeid','period','playid','secondsRemainingInPeriod','startpossessionteamid','down','yardsToGo','startYardsFromGoal','scoreDiff']]
outmat['isRush'] = label_rush[id]
outmat['pred_rush_prob'] = re_prob_rush[:,1]
outmat.to_csv('QB_rush_training_data.csv', index=False)

print("QB fitted average rush ratio: {:.2%}".format(sum(re)/len(re)) )
print("QB fitted average rush_prob ratio: {:.2%}".format(np.mean(re_prob_rush[:,1])) )
print("QB actual average rush ratio: {:.2%}\n".format(sum(label_rush[id])/sum(id)) )

In [ ]:
# scramble
model_logistic.fit(features_transformed[id], label_scramble[id])
re = model_logistic.predict(features_transformed[id])
re_prob_scramble = model_logistic.predict_proba(features_transformed[id])

print("QB fitted average scramble ratio: {:.2%}".format(sum(re)/len(re)) )
print("QB fitted average scramble_prob ratio: {:.2%}".format(np.mean(re_prob_scramble[:,1])) )
print("QB actual average scramble ratio: {:.2%}\n".format(sum(label_scramble[id])/sum(id)) )

# create output file
# outmat = pbp_df.loc[id,['season','week','gamecode','eventtypeid','period','playid','secondsRemainingInPeriod','startpossessionteamid','down','yardsToGo','startYardsFromGoal','scoreDiff']]
# outmat['isScramble'] = label_scramble[id]
# outmat['pred_scramble_prob'] = re_prob_scramble[:,1]
# outmat.to_csv('QB_scramble_training_data.csv', index=False)


# sack
model_logistic.fit(features_transformed[id], label_sack[id])
re = model_logistic.predict(features_transformed[id])
re_prob_sack = model_logistic.predict_proba(features_transformed[id])

print("QB fitted average sack ratio: {:.2%}".format(sum(re)/len(re)) )
print("QB fitted average sack_prob ratio: {:.2%}".format(np.mean(re_prob_sack[:,1])) )
print("QB actual average sack ratio: {:.2%}".format(sum(label_sack[id])/sum(id)) )

# create output file
# outmat = pbp_df.loc[id,['season','week','gamecode','eventtypeid','period','playid','secondsRemainingInPeriod','startpossessionteamid','down','yardsToGo','startYardsFromGoal','scoreDiff']]
# outmat['isSack'] = label_sack[id]
# outmat['pred_sack_prob'] = re_prob_sack[:,1]
# outmat.to_csv('QB_sack_training_data.csv', index=False)

In [ ]:
# linear regression model interpretation

import statsmodels.api as sm

mod = sm.Logit(label_pass.values+0, features_transformed, missing='drop')
res = mod.fit(method='newton', maxiter=1000)

res.summary()

In [ ]:
# feature importance study

regr = RandomForestRegressor(n_estimators=200, max_depth=40, random_state=0)
regr.fit(features_transformed, label_pass)

#cat_one_hot_fields = list(transform_pipeline.named_transformers_.cat.get_feature_names())
#feature_score = pd.DataFrame([num_fields + cat_one_hot_fields, regr.feature_importances_], 
feature_score = pd.DataFrame([feature_names, regr.feature_importances_], 
                             index=['feature','importance']).transpose()
feature_score.sort_values(by='importance',ascending=False).reset_index(drop=True)

# 3. Model performance simulator

In [ ]:
# calculate player difference: QB specific

# group by QBid: apply the function of fit to each QB

In [ ]:
# a work around -- issue with QB parsing for each play, since a snap player could be not a QB

simple_query = f'''
select playerid, firstname, lastname, positions[1].positionid positionid, positions[1].name name
from datalakefootball.players
where positions[1].positionid = 8
'''

if True:
    qb_info = cursor.execute(simple_query).as_pandas()
    print(qb_info.info())
else:
    print("Failed to query!")

In [ ]:
# the weight decay factor
#  alpha = 0.9995
#  alpha_team = 0.99

#import time

def playerRatesTesting(trainSeason_s, trainSeason_e, testSeason, features, label, pbp_df, playerids, 
                       model, alpha, alpha_team, playsThreshold, predictProb):
    playerTrain = []
    playerTest = []

    weeks = pbp_df[pbp_df.season==testSeason].week.unique()

    ### the back-testing is carried out one week at a time
    for week in weeks:
        
        id = (pbp_df.season==testSeason) & (pbp_df.week==week)
        id = id.tolist()
        test_label = label[id].reset_index(drop=True)
        test_features = features[id].reset_index(drop=True)
        test_players = pbp_df.playerid[id]

        test_player_df = pbp_df.loc[id, ['week','playerid','defenseTeam']]
        test_player_df.drop_duplicates(inplace=True)

        id = ((pbp_df.season==trainSeason_s) & (pbp_df.week >= week)) | ((pbp_df.season>trainSeason_s) & (pbp_df.season<=trainSeason_e)) | ((pbp_df.season==testSeason) & (pbp_df.week < week))
        id = id.tolist()
        train_label = label[id].reset_index(drop=True)
        train_features = features[id].reset_index(drop=True)
        train_players = pbp_df.playerid[id]
        train_weights = np.array([alpha] * np.sum(id)) # be careful data type is very important!
        
        ### fit the model
        model = model.fit(train_features, train_label)
        
        if predictProb:
            pred = model.predict_proba(train_features)[:,1]
        else:
            pred = model.predict(train_features)

        ### team_level results    
        team_df = pd.DataFrame( { 'season':pbp_df.season[id], 'week':pbp_df.week[id], 'defenseTeam':pbp_df.defenseTeam[id], 'team_pred':pred, 'team_obs':label[id] } )
        gd = team_df.groupby(by=['season','week','defenseTeam'], as_index=False)
        team_df = gd.mean()

        team_df['weights'] = [alpha_team] * team_df.shape[0]

        team_means = []
        for team in np.unique(team_df.defenseTeam):
            id = (team_df.defenseTeam == team).tolist()
            weights = np.cumprod([alpha_team] * np.sum(id))[::-1]
            team_pred = np.sum(team_df.team_pred[id] * weights) / np.sum(weights)
            team_obs  = np.sum(team_df.team_obs[id]  * weights) / np.sum(weights)
            team_means.append([team, team_pred, team_obs])
            
        team_df = pd.DataFrame(team_means, columns=['defenseTeam','team_pred','team_obs'])

        ### player_level results
        players = train_players[~train_players.isna()].unique()
        
        if len(playerids) > 0:
            players = [player for player in players if player in playerids]

        #### determine weights for each play and calculate the weighted mean over all players
        for player in players:
            id = np.array(train_players) == player
            playerWeights = np.cumprod([alpha] * np.sum(id))[::-1]
            train_weights[id] = playerWeights
        

        ### mean over all players
        all_mean = np.sum(pred * train_weights) / np.sum(train_weights)

        print(week, all_mean, pred.mean(), len(train_label), len(train_players), train_features.shape, len(test_label), len(test_players), test_features.shape)
        
        ### player_level adjustment
        for player in players:
            id = np.array(train_players) == player
            playerPred = pred[id]
            obs  = train_label[id]
            playerWeights = train_weights[id]

            playerPred_mean = np.sum(playerPred * playerWeights) / np.sum(playerWeights)
            obs_mean = np.sum(obs * playerWeights) / np.sum(playerWeights)
            obs_simple_mean = obs.mean()

            diff = obs_mean - playerPred_mean

            onePlayerEntry = pd.DataFrame([[week, player, np.sum(id), all_mean, playerPred_mean, obs_mean, obs_simple_mean, diff, all_mean + diff]],
                                     columns = ['week', 'playerid', 'trainPlayNum', 'trainAllPredictedMean', 'trainPlayerPredictedMean', 'trainPlayerObservedMean', 
                                                'trainPlayerSimpleMean', 'diff', 'predict'])

            # for rookie player, adjust the prediction if the play # is too small. 50 is too large
            if onePlayerEntry.loc[0, 'trainPlayNum'] < playsThreshold:
                onePlayerEntry.loc[0, 'predict'] = onePlayerEntry.loc[0, 'trainAllPredictedMean']

            playerTrain.append(onePlayerEntry)

        ### testing data ###
        players_2 = test_players[~test_players.isna()].unique()
        players_2 = [player for player in players_2 if player in players]

        for player in players_2:
            id = np.array(test_players) == player
            test_obs = test_label[id].mean()
            
            onePlayerEntry = pd.DataFrame([[week, player, np.sum(id), test_obs]], columns=['week', 'playerid', 'testPlayNum', 'testPlayerObservedMean'])
            onePlayerEntry = pd.merge(onePlayerEntry, test_player_df, on=['week','playerid'], how='left')
            onePlayerEntry = pd.merge(onePlayerEntry, team_df, on=['defenseTeam'], how='left')
            playerTest.append(onePlayerEntry)
            
        #if week > 0:
        #    break
        break
              
    ### combine outputs
    player_pred_df = pd.concat(playerTrain, ignore_index=True)

    playerTest_df = pd.concat(playerTest, ignore_index=True)

    player_df = pd.merge(player_pred_df, playerTest_df, on=['week', 'playerid'], how='right')

    ### add defense team adjustment
    player_df['predict_2'] = player_df.predict + (player_df.team_obs - player_df.team_pred)

    print(player_pred_df.shape, playerTest_df.shape, player_df.shape)
    
    return(player_df)

In [ ]:
QB_pass = playerRatesTesting(2015, 2018, 2019, features_transformed_2, label_pass, pbp_df, qb_info.playerid.tolist(), model_RF, 
                             0.9995, 0.99, 10, predictProb=True)
QB_pass.to_csv('QB_pass_testing_data.csv', index=False)

# MSE
print('{:.5f} {:.5f}'.format(np.mean((QB_pass.trainPlayerSimpleMean - QB_pass.testPlayerObservedMean)**2), np.mean((QB_pass.predict_2 - QB_pass.testPlayerObservedMean)**2)) )

# MAE
print('{:.4f} {:.4f}'.format(np.mean(abs(QB_pass.trainPlayerSimpleMean - QB_pass.testPlayerObservedMean)), np.mean(abs(QB_pass.predict_2 - QB_pass.testPlayerObservedMean))) )

In [ ]:
QB_rush = playerRatesTesting(2015, 2018, 2019, features_transformed, label_rush, pbp_df, qb_info.playerid.tolist(), model_RF, 
                             0.9995, 0.99, 10, predictProb=True)
QB_rush.to_csv('QB_rush_testing_data.csv', index=False)

# MSE
print('{:.5f} {:.5f}'.format(np.mean((QB_rush.trainPlayerSimpleMean - QB_rush.testPlayerObservedMean)**2), np.mean((QB_rush.predict_2 - QB_rush.testPlayerObservedMean)**2)) )

# MAE
print('{:.4f} {:.4f}'.format(np.mean(abs(QB_rush.trainPlayerSimpleMean - QB_rush.testPlayerObservedMean)), np.mean(abs(QB_rush.predict_2 - QB_rush.testPlayerObservedMean))) )

In [ ]:
QB_scramble = playerRatesTesting(2015, 2018, 2019, features_transformed, label_scramble, pbp_df, qb_info.playerid.tolist(), model_RF, 
                                 0.9995, 0.99, 10, predictProb=True)
QB_scramble.to_csv('QB_scramble_testing_data.csv', index=False)

# MSE
print('{:.5f} {:.5f}'.format(np.mean((QB_scramble.trainPlayerSimpleMean - QB_scramble.testPlayerObservedMean)**2), np.mean((QB_scramble.predict_2 - QB_scramble.testPlayerObservedMean)**2)) )

# MAE
print('{:.4f} {:.4f}'.format(np.mean(abs(QB_scramble.trainPlayerSimpleMean - QB_scramble.testPlayerObservedMean)), np.mean(abs(QB_scramble.predict_2 - QB_scramble.testPlayerObservedMean))) )

In [ ]:
QB_sack = playerRatesTesting(2015, 2018, 2019, features_transformed, label_sack, pbp_df, qb_info.playerid.tolist(), model_RF, 
                             0.9995, 0.99, 10, predictProb=True)
QB_sack.to_csv('QB_sack_testing_data.csv', index=False)

# MSE
print('{:.5f} {:.5f}'.format(np.mean((QB_sack.trainPlayerSimpleMean - QB_sack.testPlayerObservedMean)**2), np.mean((QB_sack.predict_2 - QB_sack.testPlayerObservedMean)**2)) )

# MAE
print('{:.4f} {:.4f}'.format(np.mean(abs(QB_sack.trainPlayerSimpleMean - QB_sack.testPlayerObservedMean)), np.mean(abs(QB_sack.predict_2 - QB_sack.testPlayerObservedMean))) )

In [ ]:
# scatter plot between observation and prediction

plt.figure(figsize=[9,9])
plt.plot(QB_pass.testPlayerObservedMean, QB_pass.predict_2, 'o')
plt.xlabel('QB pass %')
plt.xlim(-0.05, 1.05)
plt.ylabel('Prediction')
plt.ylim(-0.05, 1.05)
plt.plot( [-0.05,1.05],[-0.05,1.05] )
plt.show()

# 4. MLFlow logging

In [ ]:
mlflow.set_experiment('NFL-QB-rates-pbp')

from sklearn.metrics import precision_score, recall_score, f1_score

with mlflow.start_run():

    id_train = (pbp_df.season <= 2018).tolist()
    id_test =  (pbp_df.season == 2019).tolist()
    
    X_train = features_transformed_2[id_train]
    y_train = label_pass[id_train]
    X_test = features_transformed_2[id_test]
    y_test = label_pass[id_test]
    
    model = model_RF
    model.fit(X_train, y_train)
    
    # get_params() returns a dictionary of {param_name: param_value} pairs.
    model_params = model.get_params()

    # Luckily, this is the exact format that mlflow's log_params() function takes!
    mlflow.log_params(model_params)
    
    # Let's now evaluate our model's performance and log those metrics
    metrics = {
        'train_recall': recall_score(y_train, model.predict(X_train), average='macro'),
        'test_recall':  recall_score(y_test,   model.predict(X_test),  average='macro'),
        #'train_precision': precision_score(y_train, model.predict(X_train), average='macro'),
        #'test_precision': precision_score(y_test, model.predict(X_test), average='macro'),
        #'train_f1': f1_score(y_train, model.predict(X_train), average='macro'),
        #'test_f1': f1_score(y_test, model.predict(X_test), average='macro'),
    }
    print(metrics)
    mlflow.log_metrics(metrics)
    
    # log model: the mlflow.sklearn module (https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html)
    # provides lots of helpful tools for interacting with sklearn models. The function log_model() autoamtically
    # creates an artifact from your trained sklearn model with all of the necessary info to reproduce your model.
    mlflow.sklearn.log_model(model, '')
    
    mlflow.end_run()

In [ ]:
len(id_train)
features_transformed_2.shape

# 5. Multi-nomial classification

## 5.1 Label / Feature creation

In [ ]:
id_pass = pbp_df.playtypeid.isin([1,2,9])

id_rush = pbp_df.playtypeid.isin([4]) & (pbp_df.handofftype!='Q') & (pbp_df.rolloutlocation=='N')

id_scramble = pbp_df.playtypeid.isin([4]) & (pbp_df.handofftype=='Q') & (pbp_df.rolloutlocation=='N')

id_sack = pbp_df.playtypeid.isin([3])

label = np.array([0] * len(id_pass))
label[id_pass]     = 1
label[id_rush]     = 2
label[id_scramble] = 3
label[id_sack]     = 4

sum(label==0)

id_train = (pbp_df.season <= 2019).tolist()
id_test =  (pbp_df.season == 2019).tolist()

X_train = features_transformed[id_train]
y_train = label[id_train]

In [ ]:
X_train.shape

## 5.2 Model study

In [ ]:
from sklearn.model_selection import cross_val_predict

# multinomial classification model - logistic regression
model = LogisticRegression(multi_class="multinomial", solver='lbfgs', C=10, fit_intercept=True, max_iter=10000, tol=1e-4, random_state=42, class_weight=None)

y_train_pred = cross_val_predict(model, X_train, y_train, cv=5, method='predict_proba')

# binomial classification model
model = LogisticRegression(solver='lbfgs', C=10, fit_intercept=True, max_iter=10000, tol=1e-4, random_state=42, class_weight=None)
pass_train_pred = cross_val_predict(model, X_train, label_pass[id_train], cv=5, method='predict_proba')

rush_train_pred = cross_val_predict(model, X_train, label_rush[id_train], cv=5, method='predict_proba')

scramble_train_pred = cross_val_predict(model, X_train, label_scramble[id_train], cv=5, method='predict_proba')

sack_train_pred = cross_val_predict(model, X_train, label_sack[id_train], cv=5, method='predict_proba')

In [ ]:
# given this sklearn is an old version, roc for multi-nomial classification is not ready yet!

from sklearn.metrics import roc_auc_score

print(roc_auc_score(label_pass[id_train], pass_train_pred[:,1]))
print(roc_auc_score(label_pass[id_train], y_train_pred[:,1]), '\n')

print(roc_auc_score(label_rush[id_train], rush_train_pred[:,1]))
print(roc_auc_score(label_rush[id_train], y_train_pred[:,2]), '\n')

print(roc_auc_score(label_scramble[id_train], scramble_train_pred[:,1]))
print(roc_auc_score(label_scramble[id_train], y_train_pred[:,3]), '\n')

print(roc_auc_score(label_sack[id_train], sack_train_pred[:,1]))
print(roc_auc_score(label_sack[id_train], y_train_pred[:,4]))

In [ ]:
# multinomial classification model - random forest
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight=None)

y_train_pred = cross_val_predict(model, X_train, y_train, cv=5, method='predict_proba')

# binomial classification model
pass_train_pred = cross_val_predict(model, X_train, label_pass[id_train], cv=5, method='predict_proba')

rush_train_pred = cross_val_predict(model, X_train, label_rush[id_train], cv=5, method='predict_proba')

scramble_train_pred = cross_val_predict(model, X_train, label_scramble[id_train], cv=5, method='predict_proba')

sack_train_pred = cross_val_predict(model, X_train, label_sack[id_train], cv=5, method='predict_proba')

In [ ]:
print(roc_auc_score(label_pass[id_train], pass_train_pred[:,1]))
print(roc_auc_score(label_pass[id_train], y_train_pred[:,1]), '\n')

print(roc_auc_score(label_rush[id_train], rush_train_pred[:,1]))
print(roc_auc_score(label_rush[id_train], y_train_pred[:,2]), '\n')

print(roc_auc_score(label_scramble[id_train], scramble_train_pred[:,1]))
print(roc_auc_score(label_scramble[id_train], y_train_pred[:,3]), '\n')

print(roc_auc_score(label_sack[id_train], sack_train_pred[:,1]))
print(roc_auc_score(label_sack[id_train], y_train_pred[:,4]))

In [ ]:
# create output file
outmat = pbp_df.loc[id_train, ['season','week','gamecode','eventtypeid','period','playid','secondsRemainingInPeriod','startpossessionteamid','down','yardsToGo','startYardsFromGoal','scoreDiff']]
outmat['label']  = y_train
outmat['b_pass'] = pass_train_pred[:,1]
outmat['b_rush'] = rush_train_pred[:,1]
outmat['b_scramble'] = scramble_train_pred[:,1]
outmat['b_sack'] = sack_train_pred[:,1]
m_mat = pd.DataFrame(y_train_pred[:, 1:], columns=['m_pass','m_rush','m_scramble','m_sack'])

outmat = pd.concat([outmat, m_mat], axis=1)
# outmat['isScramble'] = label_scramble[id]
# outmat['pred_scramble_prob'] = re_prob_scramble[:,1]
outmat.to_csv('QB_rate_comparison_2.csv', index=False)

In [ ]:
outmat.head()

In [ ]:
id = (outmat.yardsToGo <= 1.1) & (outmat.down == 1) #& (~outmat.m_pass.isna())
isPass = outmat.label == 1.0
print(isPass[id].mean(), outmat[id].b_pass.mean(), outmat[id].m_pass.mean())
outmat[id]

isSack = outmat.label == 4.0
print(isSack.mean(), outmat[isSack].b_sack.mean(), outmat[isSack].m_sack.mean())

isScramble = outmat.label == 3.0
print(isScramble.mean(), outmat[isScramble].b_sack.mean(), outmat[isScramble].m_sack.mean())

isRush = outmat.label == 2.0
print(isRush.mean(), outmat[isRush].b_rush.mean(), outmat[isRush].m_rush.mean())

isPass = outmat.label == 1.0
print(isPass.mean(), outmat[isPass].b_pass.mean(), outmat[isPass].m_pass.mean())

In [ ]:
plt.scatter(outmat[id].b_pass, outmat[id].m_pass)